In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda"  # the device to load the model onto

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mixtral-8x7B-v0.1",
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [5]:
activations = {}


def get_expert_activations(layer_name):
    def hook(model, input, output):
        activations[layer_name] = output.detach()

    return hook


layer_num = 0
layer_name = f"h.{layer_num}.mlp"
model.transformer.h[layer_num].mlp.register_forward_hook(
    get_expert_activations(layer_name)
)

prompt = "My favourite condiment is"
with torch.no_grad():
    model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
    model.to(device)
    model(**model_inputs)

moe_activations = activations[layer_name]
expert_idx = 0
expert_activations = moe_activations[:, :, expert_idx, :]

AttributeError: 'MixtralForCausalLM' object has no attribute 'transformer'